In [23]:
import pandas as pd
from pathlib import Path
import os
os.chdir('./data-prod')
os.getcwd()

'/Users/deepakm/mywork/data-prod'

In [24]:
datapath = Path('./data/json150')
print(datapath)
file_list= list(datapath.glob('*.json'))
file_list[0]

data/json150


PosixPath('data/json150/crhg_2023_Dec_04_09_15_33_jsonfile.json')

In [25]:
import json
json_data=[]
bad_file=[]
for file in file_list:
    try:
        # print(file)
        f = open(file)
        data = json.load(f)
        json_data.append(data)
    except:
        bad_file.append(file)
    # json_data.append(data)
    f.close()
# json_data

print(len(json_data))
print(len(bad_file))

137
13


In [26]:
df=pd.json_normalize(json_data)
df['Timestamp']=df['Timestamp'].str.replace("_","-",2)
df['Timestamp']=df['Timestamp'].str.replace("_"," ",1)
df['Timestamp']=df['Timestamp'].str.replace("_",":",2)
df['Time_Start']=pd.to_datetime(df['Timestamp'],format='mixed') #,errors='coerce')
df['CallerContent']=df.loc[:,(df.columns.str.endswith('CallerContent'))].apply(lambda x: ' '.join(x.dropna().astype(str).values),axis=1)
df['ReceiverContent']=df.loc[:,(df.columns.str.endswith('ReceiverContent'))].apply(lambda x: ' '.join(x.dropna().astype(str).values),axis=1)

print(df.isna().sum())
print(df[['CallerContent','ReceiverContent']])
df.sample(10)

Metadata                                  24
Caller                                    19
Receiver                                   2
Timestamp                                  2
CallContent.Seq1.CallerContent            20
CallContent.Seq1.ReceiverContent           3
CallContent.Seq1.Timestamp                 3
CallContent.Seq2.CallerContent             4
CallContent.Seq2.ReceiverContent          11
CallContent.Seq2.Timestamp                 4
CallContent.Seq3.CallerContent            12
CallContent.Seq3.ReceiverContent          11
CallContent.Seq3.Timestamp                 6
CallContent.Seq4.CallerContent            25
CallContent.Seq4.ReceiverContent          26
CallContent.Seq4.Timestamp                18
CallContent.Seq5.CallerContent            36
CallContent.Seq5.ReceiverContent          35
CallContent.Seq5.Timestamp                27
CallContent.Seq6.CallerContent            43
CallContent.Seq6.ReceiverContent          41
CallContent.Seq6.Timestamp                37
CallConten

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,CallContent.Seq3.CallerContent,CallContent.Seq3.ReceiverContent,CallContent.Seq3.Timestamp,CallContent.Seq4.CallerContent,CallContent.Seq4.ReceiverContent,CallContent.Seq4.Timestamp,CallContent.Seq5.CallerContent,CallContent.Seq5.ReceiverContent,CallContent.Seq5.Timestamp,CallContent.Seq6.CallerContent,CallContent.Seq6.ReceiverContent,CallContent.Seq6.Timestamp,CallContent.Seq7.CallerContent,CallContent.Seq7.ReceiverContent,CallContent.Seq7.Timestamp,CallContent.Seq8.CallerContent,CallContent.Seq8.ReceiverContent,CallContent.Seq8.Timestamp,CallContent.Seq9.CallerContent,CallContent.Seq9.ReceiverContent,CallContent.Seq9.Timestamp,CallContent.Seq10.CallerContent,CallContent.Seq10.ReceiverContent,CallContent.Seq10.Timestamp,CallContent.Seq11.CallerContent,CallContent.Seq11.ReceiverContent,CallContent.Seq11.Timestamp,CallContent.Seq12.CallerContent,CallContent.Seq12.ReceiverContent,CallContent.Seq12.Timestamp,CallContent.Seq13.CallerContent,CallContent.Seq13.ReceiverContent,CallContent.Seq13.Timestamp,CallContent.Seq14.CallerContent,CallContent.Seq14.ReceiverContent,CallContent.Seq14.Timestamp,CallContent.Seq15.CallerContent,CallContent.Seq15.ReceiverContent,CallContent.Seq15.Timestamp,CallContent.Seq16.CallerContent,CallContent.Seq16.ReceiverContent,CallContent.Seq16.Timestamp,CallContent.Seq17.CallerContent,CallContent.Seq17.ReceiverContent,CallContent.Seq17.Timestamp,CallContent.Seq18.CallerContent,CallContent.Seq18.ReceiverContent,CallContent.Seq18.Timestamp,CallContent.Seq19.CallerContent,CallContent.Seq19.ReceiverContent,CallContent.Seq19.Timestamp,CallContent.Seq20.CallerContent,CallContent.Seq20.ReceiverContent,CallContent.Seq20.Timestamp,CallContent.Seq21.CallerContent,CallContent.Seq21.ReceiverContent,CallContent.Seq21.Timestamp,CallContent.Seq22.CallerContent,CallContent.Seq22.ReceiverContent,CallContent.Seq22.Timestamp,CallContent.Seq23.CallerContent,CallContent.Seq23.ReceiverContent,CallContent.Seq23.Timestamp,CallContent.Seq24.CallerContent,CallContent.Seq24.ReceiverContent,CallContent.Seq24.Timestamp,CallContent.Seq25.CallerContent,CallContent.Seq25.ReceiverContent,CallContent.Seq25.Timestamp,CallContent.Seq26.CallerContent,CallContent.Seq26.ReceiverContent,CallContent.Seq26.Timestamp,CallContent.Seq27.CallerContent,CallContent.Seq27.ReceiverContent,CallContent.Seq27.Timestamp,CallContent.Seq28.CallerContent,CallContent.Seq28.ReceiverContent,CallContent.Seq28.Timestamp,CallContent.Seq29.CallerContent,CallContent.Seq29.ReceiverContent,CallContent.Seq29.Timestamp,CallContent.Seq30.CallerContent,CallContent.Seq30.ReceiverContent,CallContent.Seq30.Timestamp,Conversation.Participants,Conversation.Messages,CallContent.Seq7.OtherSpeakerContent,CallContent.Seq9.OtherSpeakerContent,CallContent.Seq11.OtherSpeakerContent,CallContent.Seq12.OtherSpeakerContent,CallContent.Seq31.CallerContent,CallContent.Seq31.ReceiverContent,CallContent.Seq31.Timestamp,CallContent.Seq32.CallerContent,CallContent.Seq32.ReceiverContent,CallContent.Seq32.Timestamp,CallContent.Seq33.CallerContent,CallContent.Seq33.ReceiverContent,CallContent.Seq33.Timestamp,CallContent.Seq4.OtherSpeakerContent,Time,Conversation,CallContent.Seq5.BotContent,CallContent.Seq3.OtherSpeakerContent,Content,Time_Start,CallerContent,ReceiverContent
46,value,Nathaniel James,Dominic,2023-10-02 13:45:05,Hi. I'm trying to get a referral to a sleep cl...,Good morning. Thanks for calling Crofton Famil...,2023-10-02 13:45:05,Thanks.,"Okay. For the referral, I got you. Can you giv...",2023-10-02 13:45:20,Hi. They transferred me to you for getting a r...,Your conversation is being recorded. Good afte...,2023-10-02 13:45:39,Nathaniel James.,Okay. Hold on one second. Who's the patient?,2023-10-02 13:45:43,"July 10, 1991.",And your patient here?,2023-10-02 13:46:01,"Maybe 

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Columns: 118 entries, Metadata to ReceiverContent
dtypes: datetime64[ns](1), object(117)
memory usage: 126.4+ KB


In [28]:
print(df.Caller.value_counts())
print(df.Receiver.value_counts())


Caller
No Name                                              16
Mary Yonkey                                           2
Emily                                                 2
Rosemary                                              2
Shirley Jacobus                                       2
Jim Neiman                                            2
Ken Brown                                             2
Ashley                                                2
Elizabeth                                             1
Tommy                                                 1
Cynthia Parks McDaniel                                1
Wes Henderson                                         1
Ann Curran                                            1
Megan                                                 1
Iron Name (calling on behalf of his wife, Cheryl)     1
Mitchie                                               1
Cindy Raz                                             1
Brian Fryer                              

In [29]:
print(len (df.columns),df.columns)

118 Index(['Metadata', 'Caller', 'Receiver', 'Timestamp', 'CallContent.Seq1.CallerContent', 'CallContent.Seq1.ReceiverContent', 'CallContent.Seq1.Timestamp', 'CallContent.Seq2.CallerContent', 'CallContent.Seq2.ReceiverContent', 'CallContent.Seq2.Timestamp',
       ...
       'CallContent.Seq33.Timestamp', 'CallContent.Seq4.OtherSpeakerContent', 'Time', 'Conversation', 'CallContent.Seq5.BotContent', 'CallContent.Seq3.OtherSpeakerContent', 'Content', 'Time_Start', 'CallerContent', 'ReceiverContent'], dtype='object', length=118)


In [30]:
# pip install transformers
from transformers import pipeline
sentiment_model = pipeline("sentiment-analysis", truncation=True, max_length=512)
data = ["I love you", "I hate you"]
sentiment_model(data)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [31]:
# Sentiment Analysis of Caller Data
df['sent_model']= df['CallerContent'].apply(sentiment_model) 
df['Caller_Sentiment']= df['sent_model'].apply(lambda s: s[0]['label'])
df['Caller_Sentiment_Score']= df['sent_model'].apply(lambda s: s[0]['score'])
print(df[['CallerContent','Caller_Sentiment','Caller_Sentiment_Score']])

                                         CallerContent Caller_Sentiment  Caller_Sentiment_Score
0    Hi, my name is Carol Julian, and I was dischar...         POSITIVE                0.948155
1    Yes, I'm returning the call. I think they were...         NEGATIVE                0.997470
2    Hi, this is Charlotte Baker. Doctor, okay, I'm...         NEGATIVE                0.998663
3    last known thanks for calling corruption from ...         NEGATIVE                0.991963
4    Oh, yes. I'd like to make a couple of appointm...         NEGATIVE                0.921696
5    Hey Kumar, good morning. My name is Robert. I'...         POSITIVE                0.998686
6    hi good morning I'm a patient of dr. Williams ...         NEGATIVE                0.996768
7    Hi, this is Anthony Figlio. See, I was wonderi...         POSITIVE                0.998816
8    I need to schedule an appointment with Dr. Wil...         NEGATIVE                0.998505
9    Yeah, Dominic, I'd like to make an 

In [32]:
# model= "pysentimiento/robertuito-sentiment-analysis"
model="bhadresh-savani/distilbert-base-uncased-emotion"
emotion_model = pipeline(model=model, truncation=True, max_length=512)
emotion_model(data)


[{'label': 'love', 'score': 0.9608992338180542},
 {'label': 'anger', 'score': 0.826848566532135}]

In [33]:
# Sentiment Analysis of Caller Data
df['emot_model']= df['CallerContent'].apply(emotion_model) 
df['Caller_Emotion']= df['emot_model'].apply(lambda s: s[0]['label'])
df['Caller_Emotion_Score']= df['emot_model'].apply(lambda s: s[0]['score'])
print(df[['CallerContent','Caller_Emotion','Caller_Emotion_Score']])

                                         CallerContent Caller_Emotion  Caller_Emotion_Score
0    Hi, my name is Carol Julian, and I was dischar...            joy              0.963799
1    Yes, I'm returning the call. I think they were...            joy              0.466442
2    Hi, this is Charlotte Baker. Doctor, okay, I'm...            joy              0.948941
3    last known thanks for calling corruption from ...            joy              0.985529
4    Oh, yes. I'd like to make a couple of appointm...            joy              0.993119
5    Hey Kumar, good morning. My name is Robert. I'...            joy              0.997776
6    hi good morning I'm a patient of dr. Williams ...            joy              0.998790
7    Hi, this is Anthony Figlio. See, I was wonderi...            joy              0.995932
8    I need to schedule an appointment with Dr. Wil...            joy              0.998647
9    Yeah, Dominic, I'd like to make an appointment...            joy           

In [34]:
df.head()

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,CallContent.Seq3.CallerContent,CallContent.Seq3.ReceiverContent,CallContent.Seq3.Timestamp,CallContent.Seq4.CallerContent,CallContent.Seq4.ReceiverContent,CallContent.Seq4.Timestamp,CallContent.Seq5.CallerContent,CallContent.Seq5.ReceiverContent,CallContent.Seq5.Timestamp,CallContent.Seq6.CallerContent,CallContent.Seq6.ReceiverContent,CallContent.Seq6.Timestamp,CallContent.Seq7.CallerContent,CallContent.Seq7.ReceiverContent,CallContent.Seq7.Timestamp,CallContent.Seq8.CallerContent,CallContent.Seq8.ReceiverContent,CallContent.Seq8.Timestamp,CallContent.Seq9.CallerContent,CallContent.Seq9.ReceiverContent,CallContent.Seq9.Timestamp,CallContent.Seq10.CallerContent,CallContent.Seq10.ReceiverContent,CallContent.Seq10.Timestamp,CallContent.Seq11.CallerContent,CallContent.Seq11.ReceiverContent,CallContent.Seq11.Timestamp,CallContent.Seq12.CallerContent,CallContent.Seq12.ReceiverContent,CallContent.Seq12.Timestamp,CallContent.Seq13.CallerContent,CallContent.Seq13.ReceiverContent,CallContent.Seq13.Timestamp,CallContent.Seq14.CallerContent,CallContent.Seq14.ReceiverContent,CallContent.Seq14.Timestamp,CallContent.Seq15.CallerContent,CallContent.Seq15.ReceiverContent,CallContent.Seq15.Timestamp,CallContent.Seq16.CallerContent,CallContent.Seq16.ReceiverContent,CallContent.Seq16.Timestamp,CallContent.Seq17.CallerContent,CallContent.Seq17.ReceiverContent,CallContent.Seq17.Timestamp,CallContent.Seq18.CallerContent,CallContent.Seq18.ReceiverContent,CallContent.Seq18.Timestamp,CallContent.Seq19.CallerContent,CallContent.Seq19.ReceiverContent,CallContent.Seq19.Timestamp,CallContent.Seq20.CallerContent,CallContent.Seq20.ReceiverContent,CallContent.Seq20.Timestamp,CallContent.Seq21.CallerContent,CallContent.Seq21.ReceiverContent,CallContent.Seq21.Timestamp,CallContent.Seq22.CallerContent,CallContent.Seq22.ReceiverContent,CallContent.Seq22.Timestamp,CallContent.Seq23.CallerContent,CallContent.Seq23.ReceiverContent,CallContent.Seq23.Timestamp,CallContent.Seq24.CallerContent,CallContent.Seq24.ReceiverContent,CallContent.Seq24.Timestamp,CallContent.Seq25.CallerContent,CallContent.Seq25.ReceiverContent,CallContent.Seq25.Timestamp,CallContent.Seq26.CallerContent,CallContent.Seq26.ReceiverContent,CallContent.Seq26.Timestamp,CallContent.Seq27.CallerContent,CallContent.Seq27.ReceiverContent,CallContent.Seq27.Timestamp,CallContent.Seq28.CallerContent,CallContent.Seq28.ReceiverContent,CallContent.Seq28.Timestamp,CallContent.Seq29.CallerContent,CallContent.Seq29.ReceiverContent,CallContent.Seq29.Timestamp,CallContent.Seq30.CallerContent,CallContent.Seq30.ReceiverContent,CallContent.Seq30.Timestamp,Conversation.Participants,Conversation.Messages,CallContent.Seq7.OtherSpeakerContent,CallContent.Seq9.OtherSpeakerContent,CallContent.Seq11.OtherSpeakerContent,CallContent.Seq12.OtherSpeakerContent,CallContent.Seq31.CallerContent,CallContent.Seq31.ReceiverContent,CallContent.Seq31.Timestamp,CallContent.Seq32.CallerContent,CallContent.Seq32.ReceiverContent,CallContent.Seq32.Timestamp,CallContent.Seq33.CallerContent,CallContent.Seq33.ReceiverContent,CallContent.Seq33.Timestamp,CallContent.Seq4.OtherSpeakerContent,Time,Conversation,CallContent.Seq5.BotContent,CallContent.Seq3.OtherSpeakerContent,Content,Time_Start,CallerContent,ReceiverContent,sent_model,Caller_Sentiment,Caller_Sentiment_Score,emot_model,Caller_Emotion,Caller_Emotion_Score
0,value,Carol Julian,Dominic,2023-12-04 09:15:33,"Hi, my name is Carol Julian, and I was dischar...",Good morning. Thank you for calling Crofton Fa...,2023-12-04 09:15:33,Okay.,"Let me check the next available slot for you, ...",2023-12-04 09:15:36,Okay.,"Also, slots are already booked for the month o...",2023-12-04 09:15:43,"No. Okay, they want me to follow up within a w...","Okay, regarding that, I need to transfer you

In [ ]:
tmp=pd.read_json(file).reset_index()
tmp['Timestamp']=pd.to_datetime(tmp['Timestamp']).dt.normalize()
tmp.rename(columns={'Timestamp':'Time_Start'},inplace=True)
# tmp= pd.concat([tmp,tmp.CallContent.apply(pd.Series)])
tmp.head()

In [ ]:
tmp.info()